In [4]:
from src.backends.alternating_least_squares import AlternatingLeastSquares
from src.utils.dataset_indexer import DatasetIndexer
from src.utils.graphing import plot_distribution_histogram, plot_power_low, plot_losses, plot_rmse

In [26]:
dataset_indexer = DatasetIndexer(
    file_path="./ml-32m/ratings.csv",
    user_header="userId",
    item_header="movieId",
    rating_header="rating",
    limit=10000,
)

indexed_data = dataset_indexer.index(approximate_train_ratio=0.8)


The current implementation does not split the data into train and test sets exactly with the provided ratio. We use the provided ratio as a probability for a Bernoulli distribution to know whether the data point should be used as a training data or a test data.
Limit of lines (.i.e 10000) to index has been reached. Exiting without loading the rest... 


In [ ]:
als_backend = AlternatingLeastSquares(
    
)

In [21]:
recommender_builder = RecommenderSystemBuilder(
    backend=als_backend,
    
)

recommender = recommender_builder.build(
    
)

recommender.recommend()


Method:  numpy


KeyboardInterrupt: 